# Libraries and classes

In [21]:
from torch_lib.Dataset import *
from torch_lib.Model import Model, OrientationLoss

In [22]:
import os

import torch
import torch.nn as nn
from torch.autograd import Variable
from torchvision.models import vgg
from torch.utils import data

In [23]:
epochs = 100
batch_size = 8
alpha = 0.6
w = 0.4

params = {
    'batch_size':batch_size,
    'shuffle':True,
    'num_workers':6
}


In [25]:
datasetpath = '/home/hpahadia_3/Documents/3ddeepbox/code/3D-BoundingBox/Kitti/training'

model_path = os.path.abspath(os.getcwd()) + '/weights/'
latest_model = None
first_epoch = 0

In [26]:
# Intialize the KITTI Dataset
dataset = Dataset(datasetpath)
generator = data.DataLoader(dataset, **params)

# Initialize model
my_vgg = vgg.vgg19_bn(pretrained=True)
model = Model(features=my_vgg.features).cuda()
opt_SGD = torch.optim.SGD(model.parameters(), lr=0.0001, momentum=0.9)

In [27]:
# Loss functions for different branches
conf_loss_func = nn.CrossEntropyLoss().cuda()
dim_loss_func = nn.MSELoss().cuda()
orient_loss_func = OrientationLoss

In [28]:
if not os.path.isdir(model_path):
    os.mkdir(model_path)
else:
    try:
        for a in sorted(os.listdir(model_path)):
            if a.endswith('.pkl'):
                all_models.append(a)
        latest_model = all_models[-1]
    except:
        pass

In [29]:
if latest_model is not None:
    checkpoint = torch.load(model_path + latest_model)
    model.load_state_dict(checkpoint['model_state_dict'])
    opt_SGD.load_state_dict(checkpoint['optimizer_state_dict'])
    first_epoch = checkpoint['epoch']
    loss = checkpoint['loss']

In [30]:
loss

tensor(-0.3875, device='cuda:0', grad_fn=<AddBackward0>)

In [31]:
total_num_batches = int(len(dataset) / batch_size)
len(dataset), total_num_batches

(40570, 5071)

# Train the model

In [14]:
for epoch in range(first_epoch+1, epochs+1):
    curr_batch = 0
    passes = 0
    for local_batch, local_labels in generator:

        truth_orient = local_labels['Orientation'].float().cuda()
        truth_conf = local_labels['Confidence'].long().cuda()
        truth_dim = local_labels['Dimensions'].float().cuda()

        local_batch=local_batch.float().cuda()
        [orient, conf, dim] = model(local_batch)

        orient_loss = orient_loss_func(orient, truth_orient, truth_conf)
        dim_loss = dim_loss_func(dim, truth_dim)

        truth_conf = torch.max(truth_conf, dim=1)[1]
        conf_loss = conf_loss_func(conf, truth_conf)

        loss_theta = conf_loss + w * orient_loss
        loss = alpha * dim_loss + loss_theta

        opt_SGD.zero_grad()
        loss.backward()
        opt_SGD.step()


        if passes % 10 == 0:
            print("--- epoch %s | batch %s/%s --- [loss: %s]" %(epoch, curr_batch, total_num_batches, loss.item()))
            passes = 0

        passes += 1
        curr_batch += 1

    # save after every 10 epochs
    if epoch % 10 == 0:
        name = model_path + 'epoch_%s.pkl' % epoch
        print("====================")
        print ("Done with epoch %s!" % epoch)
        print ("Saving weights as %s ..." % name)
        torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': opt_SGD.state_dict(),
                'loss': loss
                }, name)
        print("====================")

--- epoch 21 | batch 0/5071 --- [loss: -0.39043617248535156]
--- epoch 21 | batch 10/5071 --- [loss: -0.3675423562526703]
--- epoch 21 | batch 20/5071 --- [loss: -0.3444667458534241]


KeyboardInterrupt: 